## 建模

### 依赖库导入

In [65]:
import hjson
import csv

import os
import sys

import pulp
import itertools
import pandas as pd
import scipy as sc
import numpy as np

In [66]:
constr = {}
with open("../config/constraints.hjson",'r',encoding='utf-8') as conf:
    constr = hjson.load(conf)
    # S for string
for conf in list(constr.items()):

    if conf[1] == -1:
#         print(conf)
        constr.pop(conf[0])
#         del conf
print(constr)
# with open()

OrderedDict([('main_quantity', 3), ('Std_main_ratio', 60), ('Rls_main_ratio', 10), ('Std_TFe', 58), ('Rls_TFe', 2), ('Std_SiO2', 5), ('Rls_SiO2', 0.3), ('Std_MgO', 1.6), ('Rls_MgO', 0.2), ('Std_Al2O3', 1.55), ('Rls_Al2O3', 0.55), ('Std_K2O', 0.075), ('Rls_K2O', 0.075), ('Std_FeO', 8), ('Rls_FeO', 0.5), ('Std_ZnO', 0.05), ('Rls_ZnO', 0.05), ('Std_CaOSiO2_ratio', 2.1), ('Rls_CaOSiO2_ratio', 0.2)])


In [67]:
data = []

with open("../config/mineral.csv",'r',encoding='gb2312') as form:
    datasource = csv.reader(form)
    header = []
    for d in datasource:
        if d[0] == 'Type':
            header.extend(d)
        else:
            for i in range(len(d)):
                if i>=2:
                    if d[i] == '':
                        d[i] = 0
                    else:
                        d[i] = float(d[i])
            data.append(d)
print(header)
print(list(data))

['Type', 'Name', 'TFe', 'SiO2', 'Al2O3', 'CaO', 'MgO', 'S', 'P', 'FeO', 'ZnO', 'K2O', 'LOI', 'H2O', 'UnitPrice']
[['M', 'PB粉', 61.96, 3.67, 2.28, 0.075, 0.069, 0.038, 0.101, 0, 0, 0.015, 5.28, 0, 856.37], ['M', '麦克粉', 60.86, 4.71, 2.46, 0.05, 0.07, 0.048, 0.087, 0, 0, 0.013, 5.69, 0, 828.12], ['M', '纽曼粉', 62.85, 3.74, 2.29, 0.067, 0.091, 0.025, 0.085, 0, 0, 0.0085, 4.03, 0, 846.04], ['M', '罗伊山粉', 61.2, 4.56, 2.4, 0.095, 0.098, 0.047, 0.055, 0, 0, 0.056, 4.38, 0, 828.27], ['M', '乌克兰粉', 65.57, 7.5, 0.205, 0.147, 0.317, 0.0, 0.00566, 0, 0, 0.0296, 2.84, 0, 916.44], ['M', '毛塔粉', 61.74, 8.83, 1.084, 0.2215, 0.2592, 0.0168, 0.0544, 0, 0, 0.0485, 1.0375, 0, 788.9], ['M', '印度粉', 58.15, 5.27, 6.46, 0.05, 0.05, 0.025, 0.0455, 0, 0, 0.0445, 4.91, 0, 752.59], ['H', '热返矿', 55.67, 5.82, 1.9, 9.98, 1.75, 0.35, 0.04, 0, 0, 0, 0, 0, 76.5], ['C', '冷返矿', 55.29, 6.08, 1.97, 9.98, 1.81, 0.3, 0.045, 0, 0, 0, 0, 0, 71.4], ['Z', '杂矿', 53.22, 7.27, 2.04, 8.71, 3.19, 0.158, 0.036, 0, 0, 0, 0, 0, 34.65], ['A1', 

In [68]:
# Constraints:

## STD
## 含铁合计100
## 脱水核算单位LOI

conrange = {}
for key,val in constr.items():
    if "Std" in key:
        conrange[key[4:]] = [val]
    elif 'Rls' in key:
        conrange[key[4:]].append(val)
    else:
        conrange[key] = val
for key,val in conrange.items():
    # upper,lower
    if key != 'main_quantity':
        val[0] += val[1]
        val[1] = val[0] - 2 * val[1]
        round(val[0],2)
        round(val[1],2)
# conrange[head]

print(conrange)


{'main_quantity': 3, 'main_ratio': [70, 50], 'TFe': [60, 56], 'SiO2': [5.3, 4.7], 'MgO': [1.8, 1.4], 'Al2O3': [2.1, 1.0], 'K2O': [0.15, 0.0], 'FeO': [8.5, 7.5], 'ZnO': [0.1, 0.0], 'CaOSiO2_ratio': [2.3000000000000003, 1.9000000000000004]}


In [69]:
# 脱水

print("=========脱水前==========")
print(data)

for mine in data:
    for content_index in range(2,len(mine)-3):
        # print(len(mine)-3)
        # print(mine[len(mine)-3])
        mine[content_index] = mine[content_index]/(1-mine[len(mine)-3]/100)
print("=========脱水后==========")
print(data)

=========脱水前==========
[['M', 'PB粉', 61.96, 3.67, 2.28, 0.075, 0.069, 0.038, 0.101, 0, 0, 0.015, 5.28, 0, 856.37], ['M', '麦克粉', 60.86, 4.71, 2.46, 0.05, 0.07, 0.048, 0.087, 0, 0, 0.013, 5.69, 0, 828.12], ['M', '纽曼粉', 62.85, 3.74, 2.29, 0.067, 0.091, 0.025, 0.085, 0, 0, 0.0085, 4.03, 0, 846.04], ['M', '罗伊山粉', 61.2, 4.56, 2.4, 0.095, 0.098, 0.047, 0.055, 0, 0, 0.056, 4.38, 0, 828.27], ['M', '乌克兰粉', 65.57, 7.5, 0.205, 0.147, 0.317, 0.0, 0.00566, 0, 0, 0.0296, 2.84, 0, 916.44], ['M', '毛塔粉', 61.74, 8.83, 1.084, 0.2215, 0.2592, 0.0168, 0.0544, 0, 0, 0.0485, 1.0375, 0, 788.9], ['M', '印度粉', 58.15, 5.27, 6.46, 0.05, 0.05, 0.025, 0.0455, 0, 0, 0.0445, 4.91, 0, 752.59], ['H', '热返矿', 55.67, 5.82, 1.9, 9.98, 1.75, 0.35, 0.04, 0, 0, 0, 0, 0, 76.5], ['C', '冷返矿', 55.29, 6.08, 1.97, 9.98, 1.81, 0.3, 0.045, 0, 0, 0, 0, 0, 71.4], ['Z', '杂矿', 53.22, 7.27, 2.04, 8.71, 3.19, 0.158, 0.036, 0, 0, 0, 0, 0, 34.65], ['A1', '生石灰粉', 0.0, 2.46, 0.0, 1.57, 76.36, 0.0, 0.0, 0, 0, 0, 11.81, 0, 23.205], ['A2', '高镁粉', 0

In [70]:
# 主矿表

main_mineral = []
const_content = {}
for mine in data:
    if mine[0] == 'M':
        main_mineral.append(mine[1:])
    else:
        const_content[mine[0]] = mine[1:]
print(const_content)
print(main_mineral)

{'H': ['热返矿', 55.67, 5.82, 1.9, 9.98, 1.75, 0.35, 0.04, 0.0, 0.0, 0.0, 0, 0, 76.5], 'C': ['冷返矿', 55.29, 6.08, 1.97, 9.98, 1.81, 0.3, 0.045, 0.0, 0.0, 0.0, 0, 0, 71.4], 'Z': ['杂矿', 53.22, 7.27, 2.04, 8.71, 3.19, 0.158, 0.036, 0.0, 0.0, 0.0, 0, 0, 34.65], 'A1': ['生石灰粉', 0.0, 2.7894319083796346, 0.0, 1.7802471935593605, 86.58578070075973, 0.0, 0.0, 0.0, 0.0, 0.0, 11.81, 0, 23.205], 'A2': ['高镁粉', 0.0, 8.434014869888475, 0.0, 90.56691449814126, 3.2527881040892193, 0.0, 0.0, 0.0, 0.0, 0.0, 13.92, 0, 2.745], 'J': ['焦粉', 0.00975609756097561, 60.731707317073166, 42.520325203252035, 6.097560975609756, 1.7886178861788617, 6.829268292682927, 0.0, 0.0, 0.0, 0.0, 87.7, 0, 24.654], 'B': ['煤粉', 0.011888111888111886, 52.93706293706293, 35.17482517482517, 3.8461538461538463, 2.2377622377622375, 6.573426573426572, 0.0, 0.0, 0.0, 0.0, 85.7, 0, 25.654]}
[['PB粉', 65.41385135135135, 3.8745777027027026, 2.4070945945945943, 0.07918074324324324, 0.07284628378378379, 0.04011824324324324, 0.10663006756756757, 0.0

In [71]:
import time
# VALID ASSERTION
if constr['main_quantity'] <= 6:
    # CODING
    index = 0
    Result = []
    for p in itertools.combinations(main_mineral,constr['main_quantity']):
        # print(p)
        Result.append({'NO': index, 'MAINS': [m[0] for m in p]})
        content = ['H','C','Z','A1','A2','J','B']
        TFe = {
            'H': const_content['H'][1],
            'C': const_content['C'][1],
            'Z': const_content['Z'][1],
            'A1': const_content['A1'][1],
            'A2': const_content['A2'][1],
            'J': const_content['J'][1],
            'B': const_content['B'][1]
        }
        SiO2 = {
            'H': const_content['H'][2],
            'C': const_content['C'][2],
            'Z': const_content['Z'][2],
            'A1': const_content['A1'][2],
            'A2': const_content['A2'][2],
            'J': const_content['J'][2],
            'B': const_content['B'][2]
        }
        Al2O3 = {
            'H': const_content['H'][3],
            'C': const_content['C'][3],
            'Z': const_content['Z'][3],
            'A1': const_content['A1'][3],
            'A2': const_content['A2'][3],
            'J': const_content['J'][3],
            'B': const_content['B'][3]
        }
        CaO = {
            'H': const_content['H'][4],
            'C': const_content['C'][4],
            'Z': const_content['Z'][4],
            'A1': const_content['A1'][4],
            'A2': const_content['A2'][4],
            'J': const_content['J'][4],
            'B': const_content['B'][4]
        }
        MgO = {
            'H': const_content['H'][5],
            'C': const_content['C'][5],
            'Z': const_content['Z'][5],
            'A1': const_content['A1'][5],
            'A2': const_content['A2'][5],
            'J': const_content['J'][5],
            'B': const_content['B'][5]
        }
        S = {
            'H': const_content['H'][6],
            'C': const_content['C'][6],
            'Z': const_content['Z'][6],
            'A1': const_content['A1'][6],
            'A2': const_content['A2'][6],
            'J': const_content['J'][6],
            'B': const_content['B'][6]
        }
        P = {
            'H': const_content['H'][7],
            'C': const_content['C'][7],
            'Z': const_content['Z'][7],
            'A1': const_content['A1'][7],
            'A2': const_content['A2'][7],
            'J': const_content['J'][7],
            'B': const_content['B'][7]
        }
        FeO = {
            'H': const_content['H'][8],
            'C': const_content['C'][8],
            'Z': const_content['Z'][8],
            'A1': const_content['A1'][8],
            'A2': const_content['A2'][8],
            'J': const_content['J'][8],
            'B': const_content['B'][8]
        }
        ZnO = {
            'H': const_content['H'][9],
            'C': const_content['C'][9],
            'Z': const_content['Z'][9],
            'A1': const_content['A1'][9],
            'A2': const_content['A2'][9],
            'J': const_content['J'][9],
            'B': const_content['B'][9]
        }
        K2O = {
            'H': const_content['H'][10],
            'C': const_content['C'][10],
            'Z': const_content['Z'][10],
            'A1': const_content['A1'][10],
            'A2': const_content['A2'][10],
            'J': const_content['J'][10],
            'B': const_content['B'][10]
        }
        cost = {
            'H': const_content['H'][-1],
            'C': const_content['C'][-1],
            'Z': const_content['Z'][-1],
            'A1': const_content['A1'][-1],
            'A2': const_content['A2'][-1],
            'J': const_content['J'][-1],
            'B': const_content['B'][-1]
        }
        
        for i in range(constr['main_quantity']):
            content.append(f'M{i}')
            TFe[f"M{i}"] = p[i][1]
            SiO2[f"M{i}"] = p[i][2]
            Al2O3[f"M{i}"] = p[i][3]
            CaO[f"M{i}"] = p[i][4]
            MgO[f"M{i}"] = p[i][5]
            S[f"M{i}"] = p[i][6]
            P[f"M{i}"] = p[i][7]
            FeO[f"M{i}"] = p[i][8]
            ZnO[f"M{i}"] = p[i][9]
            K2O[f"M{i}"] = p[i][10]
            cost[f"M{i}"] = p[i][-1]
        # print(content)
        # print(filename)
        # print(cost)
        recipe = pulp.LpProblem("The mineral collocation problem", pulp.LpMinimize)
        # VAR
        ingredient_vars = pulp.LpVariable.dicts("Ingr", content, 0, 1)
        # OBJ
        recipe += pulp.lpSum(cost[i]*ingredient_vars[i] for i in content)
        # CONS

        # ['Type', 'Name', 'TFe', 'SiO2', 'Al2O3', 'CaO', 'MgO', 'S', 'P', 'FeO', 'ZnO', 'K2O', 'LOI', 'H2O', 'UnitPrice']
        if "TFe" in conrange.keys():
            recipe += pulp.lpSum(TFe[i]*ingredient_vars[i] for i in content) <= conrange["TFe"][0], "TFe-upper"
            recipe += pulp.lpSum(TFe[i]*ingredient_vars[i] for i in content) >= conrange["TFe"][1], "TFe-lower"
        if "SiO2" in conrange.keys():
            recipe += pulp.lpSum(SiO2[i]*ingredient_vars[i] for i in content) <= conrange["SiO2"][0], "SiO2-upper"
            recipe += pulp.lpSum(SiO2[i]*ingredient_vars[i] for i in content) >= conrange["SiO2"][1], "SiO2-lower"
        if "Al2O3" in conrange.keys():
            recipe += pulp.lpSum(Al2O3[i]*ingredient_vars[i] for i in content) <= conrange["Al2O3"][0], "Al2O3-upper"
            recipe += pulp.lpSum(Al2O3[i]*ingredient_vars[i] for i in content) >= conrange["Al2O3"][1], "Al2O3-lower"
        if "CaO" in conrange.keys():
            recipe += pulp.lpSum(CaO[i]*ingredient_vars[i] for i in content) <= conrange["CaO"][0], "CaO-upper"
            recipe += pulp.lpSum(CaO[i]*ingredient_vars[i] for i in content) >= conrange["CaO"][1], "CaO-lower"
        if "MgO" in conrange.keys():
            recipe += pulp.lpSum(MgO[i]*ingredient_vars[i] for i in content) <= conrange["MgO"][0], "MgO-upper"
            recipe += pulp.lpSum(MgO[i]*ingredient_vars[i] for i in content) >= conrange["MgO"][1], "MgO-lower"
        if "S" in conrange.keys():
            recipe += pulp.lpSum(S[i]*ingredient_vars[i] for i in content) <= conrange["S"][0], "S-upper"
            recipe += pulp.lpSum(S[i]*ingredient_vars[i] for i in content) >= conrange["S"][1], "S-lower"
        if "P" in conrange.keys():
            recipe += pulp.lpSum(P[i]*ingredient_vars[i] for i in content) <= conrange["P"][0], "P-upper"
            recipe += pulp.lpSum(P[i]*ingredient_vars[i] for i in content) >= conrange["P"][1], "P-lower"
        if "FeO" in conrange.keys():
            recipe += pulp.lpSum(FeO[i]*ingredient_vars[i] for i in content) <= conrange["FeO"][0], "FeO-upper"
            recipe += pulp.lpSum(FeO[i]*ingredient_vars[i] for i in content) >= conrange["FeO"][1], "FeO-lower"
        if "ZnO" in conrange.keys():
            recipe += pulp.lpSum(ZnO[i]*ingredient_vars[i] for i in content) <= conrange["ZnO"][0], "ZnO-upper"
            recipe += pulp.lpSum(ZnO[i]*ingredient_vars[i] for i in content) >= conrange["ZnO"][1], "ZnO-lower"
        if "CaOSiO2" in conrange.keys():
            recipe += pulp.lpSum(CaO[i]*ingredient_vars[i] for i in content)/(pulp.lpSum(SiO2[i]*ingredient_vars[i] for i in content) <= conrange["CaOSiO2_ratio"][0]), "CaOSiO2_ratio-upper"
            recipe += pulp.lpSum(CaO[i]*ingredient_vars[i] for i in content)/(pulp.lpSum(SiO2[i]*ingredient_vars[i] for i in content) >= conrange["CaOSiO2_ratio"][1]), "CaOSiO2_ratio-lower"

        if "main_ratio" in conrange.keys():
            recipe += pulp.lpSum(ingredient_vars[i] for i in content if 'M' in i) <= conrange['main_ratio'][0], "main_ratio-upper"
            recipe += pulp.lpSum(ingredient_vars[i] for i in content if 'M' in i) >= conrange['main_ratio'][1], "main_ratio-lower"

        recipe += pulp.lpSum(ingredient_vars[i] for i in content if ('M' in i or 'H' in i or 'C' in i or 'Z' in i)) == 100
        
        
        recipe.writeLP(str(f"MODEL-{index}({time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())}).lp"))
        # recipe.writeLP("model.lp")
        recipe.solve()
        # print("Status:", pulp.LpStatus[recipe.status])
        Result[index]["STATUS"] = pulp.LpStatus[recipe.status]
        Result[index]["RESULT"] = {}
        for v in recipe.variables():
            # print(v.name, "=", v.varValue)
            Result[index]["RESULT"][v.name] = v.varValue
        index += 1
    print(Result)
else:
    print("所键入的主矿个数暂不受支持")


[{'NO': 0, 'MAINS': ['PB粉', '麦克粉', '纽曼粉'], 'STATUS': 'Infeasible', 'RESULT': {'Ingr_A1': 0.0, 'Ingr_A2': 0.0, 'Ingr_B': 0.0, 'Ingr_C': 0.0, 'Ingr_H': 0.0, 'Ingr_J': 0.0, 'Ingr_M0': 0.0, 'Ingr_M1': 50.0, 'Ingr_M2': 0.0, 'Ingr_Z': 0.0, '__dummy': None}}, {'NO': 1, 'MAINS': ['PB粉', '麦克粉', '罗伊山粉'], 'STATUS': 'Infeasible', 'RESULT': {'Ingr_A1': 0.0, 'Ingr_A2': 0.0, 'Ingr_B': 0.0, 'Ingr_C': 0.0, 'Ingr_H': 0.0, 'Ingr_J': 0.0, 'Ingr_M0': 0.0, 'Ingr_M1': -5941.4754, 'Ingr_M2': 5991.4754, 'Ingr_Z': 0.0, '__dummy': None}}, {'NO': 2, 'MAINS': ['PB粉', '麦克粉', '乌克兰粉'], 'STATUS': 'Infeasible', 'RESULT': {'Ingr_A1': 0.0, 'Ingr_A2': 0.0, 'Ingr_B': 0.0, 'Ingr_C': 0.0, 'Ingr_H': 0.0, 'Ingr_J': 0.0, 'Ingr_M0': 0.0, 'Ingr_M1': 50.0, 'Ingr_M2': 0.0, 'Ingr_Z': 0.0, '__dummy': None}}, {'NO': 3, 'MAINS': ['PB粉', '麦克粉', '毛塔粉'], 'STATUS': 'Infeasible', 'RESULT': {'Ingr_A1': 0.0, 'Ingr_A2': 0.0, 'Ingr_B': 0.0, 'Ingr_C': 0.0, 'Ingr_H': 0.0, 'Ingr_J': 0.0, 'Ingr_M0': 0.0, 'Ingr_M1': 49.0, 'Ingr_M2': 1.0, 'Ingr_Z': 0

In [72]:
# DECODER


with open(f"RESULT({time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())}).txt", 'w', encoding='utf8') as output_file:
    for plan in Result:
        # print(plan["NO"])
        output_file.write("\n============================\n")
        # output_file.write(f"报告编号：{plan["NO"]}")
        # output_file.write(f"使用主矿：{plan["MAINS"]}")
        output_file.write(f"报告编号：{plan['NO']}\n")
        output_file.write(f"使用主矿：{plan['MAINS']}\n")
        if plan["STATUS"] == "Infeasible":
            output_file.write("运行结果：无可行解\n")
        elif plan["STATUS"] == "Not Solved":
            output_file.write("运行结果：不可解\n")
        elif plan["STATUS"] == "Optimal":
            output_file.write("运行结果：最优解\n")
        elif plan["STATUS"] == "Unbounded":
            output_file.write("运行结果：命题不收敛\n")
        elif plan["STATUS"] == "Undefined":
            output_file.write("运行结果：命题未定义\n")
        else:
            output_file.write("运行结果：未知\n")
            output_file.write("计算时可能发生严重错误，请联系开发者进行处理\n")

        if plan["RESULT"] != None:
            output_file.write("\n=========配矿结果=========\n\n")
            output_file.write("\n##主矿配比##\n")
            # print(constr['main_quantity'])
            for index in range(constr['main_quantity']):
               output_file.write(f"主矿-{plan['MAINS'][index]}: {plan['RESULT'][f'Ingr_M{index}']}\n")

            output_file.write("\n##其他成分##\n")
            # H
            output_file.write(f"{const_content['H'][0]}: {plan['RESULT']['Ingr_H']}\n")
            # C
            output_file.write(f"{const_content['C'][0]}: {plan['RESULT']['Ingr_C']}\n")
            # Z
            output_file.write(f"{const_content['Z'][0]}: {plan['RESULT']['Ingr_Z']}\n")
            # A1
            output_file.write(f"{const_content['A1'][0]}: {plan['RESULT']['Ingr_A1']}\n")
            # A2
            output_file.write(f"{const_content['A2'][0]}: {plan['RESULT']['Ingr_A2']}\n")
            # JS
            output_file.write(f"{const_content['J'][0]}: {plan['RESULT']['Ingr_J']}\n")
            # B
            output_file.write(f"{const_content['B'][0]}: {plan['RESULT']['Ingr_B']}\n")

            output_file.write("\n##综合参数##\n")

            pass
        else:
            output_file.write("=========解集为空=========\n")
